# Lesson 5: PyMC3 and Bayesian Regression

## Intro to Quantified Cognition

By: Per B. Sederberg, PhD

<a href="https://colab.research.google.com/github/compmem/QuantCog/blob/2021_Spring/notebooks/05_Bayesian_Regression.ipynb"><img align="left" src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open in Colab" title="Open and Execute in Google Colaboratory"></a>

## Lesson plan

- Introduction to PyMC3
- Application to BEST
- Extension of BEST
- Introduce Bayesian regression
- Example with simulated data
- Robust regression


## Hamiltonian Monte Carlo (HMC)

Generating proposals from a random kernel can be *VERY* inefficient.

One way is to use the gradients of the posterior. An extremely popular method is the No-U-Turn (NUTS) sampler (Hoffman & Gelman, 2011).


In [ ]:
# if on Google Colab
#!pip install git+https://github.com/arviz-devs/arviz
!pip install arvis

# to retrieve the dists.py and data files
!wget https://raw.githubusercontent.com/compmem/QuantCog/2021_Spring/notebooks/dists.py

# if NOT on Google Colab and you need pymc3:
#!conda install --yes --prefix {sys.prefix} -c conda-forge pymc3

# you may also need graphviz
#conda install -c conda-forge python-graphviz

In [ ]:
# load matplotlib inline mode
%matplotlib inline

# import some useful libraries
import numpy as np                # numerical analysis linear algebra
import pandas as pd               # efficient tables
import matplotlib.pyplot as plt   # plotting
from scipy import stats

import pymc3 as pm

import dists


In [ ]:
# generate some data that may or may not be significantly different from zero
A = dists.normal(mean=0.3, std=0.5).rvs(10)

# plot it
plt.hist(A, bins='auto', density=True);

# do a quick t-test
stats.ttest_1samp(A, 0.0)

In [ ]:
# define a model
with pm.Model() as model:
    # set up the params/priors
    mu = pm.Normal('mu', A.mean(), A.std()*2.0)
    sd = pm.Uniform('sd', lower=0.01, upper=10.0)
    nu = pm.Exponential('df_minus_one', 1/29.) + 1.
    
    # build the model
    #lam = data_std**-2.
    data = pm.StudentT('data', nu=nu, mu=mu, sd=sd, observed=A)
    
    # set up some deterministic vars to keep
    effect_size = pm.Deterministic('effect_size', mu/sd)
    
    

In [ ]:
model

In [ ]:
pm.model_to_graphviz(model)

In [ ]:
with model:
    trace = pm.sample(2000, cores=2)

In [ ]:
pm.traceplot(trace);

In [ ]:
pm.plot_posterior(trace, varnames=['mu', 'sd', 'df_minus_one', 'effect_size']);

In [ ]:
pm.plot_posterior(trace, varnames=['mu', 'sd', 'effect_size'],
                  ref_val=0.0);

## Independent BEST

Let's extend the example to independent samples!!!


In [ ]:
# generate some data that may or may not be significantly different from each other
A = dists.normal(mean=0.2, std=0.5).rvs(10)
B = dists.normal(mean=0.4, std=1.0).rvs(12)

# plot it
plt.hist(A, bins='auto', alpha=0.3);
plt.hist(B, bins='auto', alpha=0.3);

# do a quick t-test
stats.ttest_ind(A, B)

In [ ]:
# first get overall mean and std
overall_mean = np.append(A, B).mean()
overall_std = np.append(A, B).std()
print('overall_mean:', overall_mean)
print('ovearll_std:', overall_std)

In [ ]:
# explore the half Cauchy prior
x = np.linspace(0, 20, 100)
plt.plot(x, dists.halfcauchy(scale=2).pdf(x))

plt.plot(x, dists.halfcauchy(scale=5).pdf(x))

In [ ]:
# define a model
with pm.Model() as model:
    # set up the params/priors
    mu_A = pm.Normal('mu_A', A.mean(), A.std()*2.0)
    sd_A = pm.HalfCauchy('sd_A', 5)
    
    mu_B = pm.Normal('mu_B', B.mean(), B.std()*2.0)
    sd_B = pm.HalfCauchy('sd_B', 5)
    
    
    nu = pm.Exponential('df_minus_one', 1/29.) + 1.
    
    # build the model
    #lam = data_std**-2.
    data_A = pm.StudentT('data_A', mu=mu_A, sd=sd_A, nu=nu, observed=A)
    data_B = pm.StudentT('data_B', mu=mu_B, sd=sd_B, nu=nu, observed=B)
    
    # set up some deterministic vars to keep
    diff_of_means = pm.Deterministic('difference of means', mu_A - mu_B)
    diff_of_stds = pm.Deterministic('difference of stds', sd_A - sd_B)
    effect_size = pm.Deterministic('effect size',
                                   diff_of_means / np.sqrt((sd_A**2 + sd_B**2) / 2))

    

In [ ]:
model

In [ ]:
pm.model_to_graphviz(model)

In [ ]:
with model:
    trace = pm.sample(2000, cores=2)

In [ ]:
pm.traceplot(trace);

In [ ]:
pm.plot_posterior(trace, varnames=['mu_A', 'mu_B', 'sd_A', 'sd_B', 'df_minus_one', 'effect size']);

In [ ]:
pm.plot_posterior(trace, varnames=['difference of means','difference of stds', 'effect size'],
                  ref_val=0.0);


## Linear Regression

One of the most common and flexible statistical approaches.

Involves building a model that can predict the dependent data ($y$) based on different combinations of independent data ($x$):

$$y = \beta_0 + \beta_1 x + \epsilon$$


In [ ]:
# generate some data with a linear trend
nsamples = 100
true_slope = 0.75
true_intercept = 1.0
true_sigma = 0.5

# uniform sampling over x
x = dists.uniform(0, 1).rvs(nsamples)

# apply noise to linear model
y_true = true_intercept + true_slope*x 
y = y_true + dists.normal(mean=0.0, std=true_sigma).rvs(nsamples)

# set the data
data = pd.DataFrame(dict(x=x, y=y))

# plot the data
plt.plot(x, y, 'o')
plt.plot(x, y_true, '-')

In [ ]:
data.head()

In [ ]:
# define a standard linear model
with pm.Model() as model:
    # set up the params/priors
    intercept = pm.Normal('intercept', 0, 20)
    slope = pm.Normal('slope', 0, 20)
    sigma = pm.HalfCauchy('sigma', 10)
    
    # calculate the line
    mu = intercept + slope * x
    
    # combine them into a linear function for the likelihood
    likelihood = pm.Normal('y', mu=mu, 
                           sd=sigma, observed=y)
    

In [ ]:
# sample the posterior
with model:
    trace = pm.sample(2000, cores=2)

In [ ]:
pm.traceplot(trace);

In [ ]:
pm.plot_posterior(trace, varnames=['intercept', 'slope', 'sigma'],
                  ref_val=0.0);

## Dealing with outliers

Sometimes data can be messy. You can either assume every observation affects the statistical inference similarly, or you can try and downplay the effect of potential outliers.

This approach is also known as robust regression.


In [ ]:
# let's add in some outliers!
x_out = np.append(x, [.08, .1, .15, .3])
y_out = np.append(y, [5.9, 3.54, 4.1, 3.2])

# plot the data
plt.plot(x_out, y_out, 'o')
plt.plot(x, y_true, '-')

In [ ]:
# define a linear model with Gaussian noise
with pm.Model() as model:
    # set up the params/priors
    intercept = pm.Normal('intercept', 0, 20)
    slope = pm.Normal('slope', 0, 20)
    sigma = pm.HalfCauchy('sigma', 10)
    
    # combine them into a linear function for the likelihood
    likelihood = pm.Normal('y_out', mu=intercept + slope * x_out, 
                           sd=sigma, observed=y_out)
    

In [ ]:
with model:
    trace = pm.sample(2000, cores=2)

In [ ]:
pm.traceplot(trace);

In [ ]:
pm.plot_posterior(trace, varnames=['intercept', 'slope', 'sigma'],
                  ref_val=[true_intercept, 0.0, true_sigma]);

In [ ]:
# let's check with the posterior predictives
lm = lambda x, samples: samples['intercept'] + x*samples['slope']

# plot the data
plt.plot(x_out, y_out, 'o')
plt.plot(x, y_true, '-')

pm.plot_posterior_predictive_glm(trace, eval=np.linspace(0, 1, 100), 
                                 lm=lm, samples=200, color="green", alpha=.15)

In [ ]:
# Can we fix it?
# define a model
with pm.Model() as model:
    # set up the params/priors
    intercept = pm.Normal('intercept', 0, 20)
    slope = pm.Normal('slope', 0, 20)
    sigma = pm.HalfCauchy('sigma', 10)
    nu = pm.Exponential('df_minus_one', 1/29.) + 1.
    
    # combine them into a robust linear function for the likelihood
    likelihood = pm.StudentT('y_out', mu=intercept + slope * x_out, 
                             sd=sigma, nu=nu, observed=y_out)
    

In [ ]:
with model:
    trace = pm.sample(2000, cores=2)

In [ ]:
pm.traceplot(trace);

In [ ]:
pm.plot_posterior(trace, varnames=['intercept', 'slope', 'sigma', 'df_minus_one'],
                  ref_val=[true_intercept, 0.0, true_sigma, 0.0]);

In [ ]:
# let's check with the posterior predictives
lm = lambda x, samples: samples['intercept'] + x*samples['slope']

# plot the data
plt.plot(x_out, y_out, 'o')
plt.plot(x, y_true, '-')

pm.plot_posterior_predictive_glm(trace, eval=np.linspace(0, 1, 100), 
                                 lm=lm, samples=200, color="green", alpha=.15)

## Assignment before next class

- I'm finally grading old assignments and posting new assignments.
- Look for the assignments posted to the assignments folder and on UVACollab

### See you next week!!!